In [1]:
from ipywidgets import widgets
from ipywidgets import Layout
from IPython import display

import numpy as np
import serial
import time
import sys
import glob
import io

ramps=False

def serial_ports():
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

def send_printer_ramps(cmd='', show=False):
    global ser_p

    ser_p.write(cmd)
    out=ser_p.readline()
    if out != '':
        if show:
            print(out)
    return out

def send_printer_marlin(cmd='', show=False):
    global ser_p
    
    if not cmd == '':
        ser_p.flushInput()
        ser_p.write(cmd)
        out = ser_p.readline().strip();
    if show:
        print(out)
    return out

def printer_init_marlin():
    global ser_p
    ser_p.write(b'G0\n')
    out=ser_p.readline().strip()
    while (out != b'ok'):
        ser_p.write(b'G0\n')
        out=ser_p.readline().strip()
    
def print_parser(cmdlines):
    for line in cmdlines.split(';'):
        if not (line== ''):
            out=(line+'\n').encode()
            if ramps:
                send_printer_ramps(out)
            else:
                send_printer_marlin(out)

def read_scale(n):
    ser_s.flushInput()
    res=[ser_s.readline() for i in range(n)]
    return [float(res[i].decode()) for i in range(len(res))]

def read_scale_times(t_s=1.):
    res=[]
    res_t=[]
    ser_s.flushInput()
    start = time.time()

    while (time.time()-start<=t_s):
        res.append(ser_s.readline())
        res_t.append(time.time()-start)
    res=[float(res[i].decode()) for i in range(len(res))]
    
    return res,res_t
    

In [12]:
%matplotlib inline
import threading
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import scipy.integrate as integrate
import pickle

g=9.81
obj_h=23. #height in mm
obj_A=np.pi*(13e-3)**2 #Area in m2
delta0=2.5 #initial displacement in mm
nn=10
rate=0.5
deltaz=1.5

    
if ramps:
    bspeed=140
else:
    bspeed=87

def start_measurement(b):
    global xdata,ydata, xnp, ynp
    deltaz=w_deltaz.value
    nn=w_nn.value
    rate=w_rate.value
    speed=w_bspeed.value*rate

    if ~ramps:
        send_printer_marlin('G1 F{:d}\r\n'.format(int(speed*deltaz)).encode())

    #generate G-code to send to printer
    cmd=''
    for i in range(nn):
        if ramps:
            cmd +='$J=G91 Z{:4.1f}F{:4.0f};'.format(-deltaz, speed)
            cmd +='$J=G91 Z{:4.1f}F{:4.0f};'.format(deltaz, speed)
        else:
            cmd +='G0 Z{:4.1f};'.format(-deltaz)
            cmd +='G0 Z{:4.1f};'.format(deltaz)

    ser_s.write(b'*')  #Tare the scale
    ser_s.flushOutput()

    time.sleep(2) #stabilize the scale

    pt = threading.Thread(target=print_parser, args=(cmd,))
    pt.start()

    ydata,xdata=read_scale_times(nn*deltaz/rate)
    xnp=np.array(xdata,dtype=np.float32)
    ynp=np.array(ydata,dtype=np.float32)

    update_plot()

def update_plot():
    global peaks, xnp, ynp
    
    peaks, _ = find_peaks(ydata, height=ynp.max()/2., distance=len(ydata)/2./nn)
    with out1:
        out1.clear_output(True)
        plt.plot(xnp, ynp)
        for i in range(len(peaks)):
            plt.text(xnp[peaks[i]], ynp[peaks[i]],str(i))
        plt.show()

def analyse(b):
    global g,obj_h,obj_A,delta0,peaks

    peak_i=w_sel_peak.value #peak position to analyse


    down_start_index=int(peaks[peak_i]+(peaks[peak_i+1]-peaks[peak_i])/2.)#find down start index
    down_end_index=peaks[peak_i+1]
    up_start_index=peaks[peak_i]
    down_stroke=ynp[down_start_index:down_end_index]
    up_stroke=ynp[up_start_index:down_start_index-1]
    down_pos=np.linspace(0,deltaz,len(down_stroke))
    up_pos=deltaz-np.linspace(0,deltaz,len(up_stroke))
    ### Calculate strain 
    max_strain,_ = integrate.quad(lambda xx: -1./xx, obj_h-delta0, obj_h-delta0-down_pos[-1])
    down_strain = np.linspace(0.,max_strain,len(down_stroke))
    up_strain = np.linspace(max_strain, 0.,len(up_stroke))

    up_stress=up_stroke*1e-3*g/obj_A
    down_stress=down_stroke*1e-3*g/obj_A

    #Simple Expression for Elastic Modulus, better by fitting a line to the center and take the slope 
    EM0=(down_stress[-1]-down_stress[0])/max_strain
    with out2:
        out2.clear_output(True)
        plt.text(0,(down_stress[-1]-down_stress[0])/2.+down_stress[0],"E={:5.1f}kPa".format(EM0/1e3))
        plt.plot(down_strain,down_stress,label='down');
        plt.plot(up_strain,up_stress,label='up')
        plt.legend()
        plt.show()

def save_measurement(b):
    global g,obj_h, obj_A, delta0, peaks, xdata, ydata, xnp, ynp
    
    peak_i=w_sel_peak.value
    deltaz=w_deltaz.value
    nn=w_nn.value
    rate=w_rate.value
    speed=w_bspeed.value*rate
    
    with open(w_filename.value, 'wb') as f: 
        pickle.dump([peak_i, g, obj_h, obj_A, delta0, peaks, xdata, ydata, xnp, ynp,\
                     deltaz, nn, rate, speed ], f)
    print('File '+w_filename.value+' saved as pkl file.')
    

def my_connect(newvalue):
    global ser_s, ser_p
    if (w_connect.value=='connect'):
        try:
            ser_s = serial.Serial(w_pscale.value, baudrate=115200, timeout=5) #scale
            ser_p = serial.Serial(w_pzaxis.value, baudrate=115200, timeout=5) #printer
            time.sleep(1)
            connected = True
            if ~ramps:
                printer_init_marlin()
                send_printer_marlin(b'M203 Z20\r\n') #increase maximum feedrate for z axis
                send_printer_marlin(b'M107\r\n') #fan off 
                send_printer_marlin(b'G91\r\n') #relative positioning
        except:
            w_connect.value='disconnect'
            connected=False
            ser_s.close()
            ser_p.close()
            print("ERROR: couldn't open serial port.")
            time.sleep(1)
            #reread serial ports
            myserialports=serial_ports()
            w_pscale.options=myserialports
            w_pzaxis.options=myserialports
            w_pscale.value=myserialports[-1]
            w_pzaxis.value=myserialports[-2]

    else:
        ser_p.close()
        ser_s.close()
        connected = False

def zaxis_ctrl(b):
    cmd=''
    if b.description=='Down 0.5 mm':
        m=-0.5
    if b.description=='Down 5 mm':
        m=-5.
    if b.description=='Up 0.5 mm':
        m=0.5
    if b.description=='Up 5 mm':
        m=5.
    if ~ramps:
        cmd='G91; G0 Z{:4.1f} F100;'.format(m)
    else:
        cmd='$J=G91 Z{:4.1f} F100;'.format(m)
    print_parser(cmd)

def approach(b):
    ser_s.write(b'*')  #Tare the scale
    ser_s.flushOutput()
    time.sleep(2) #stabilize the scale
    m=-0.1
    while True:
        if ~ramps:
            cmd='G91; G0 Z{:4.1f} F100'.format(m)
        else:
            cmd='$J=G91 Z{:4.1f} F100'.format(m)
        print_parser(cmd)
        time.sleep(.2)
        k=np.array(read_scale(10)).mean()
        w_a_label.value='Weight {:4.1f}g'.format(k)
        if (k>w_triggerw.value):
           break
        
        
try:
    myserialports
except NameError:
    myserialports=serial_ports()
       
w_pmove=[]
w_connect=widgets.ToggleButtons(description='Devices:',\
                               options=['connect', 'disconnect'], value="disconnect")
w_pscale=widgets.Dropdown(options=myserialports,\
                         value=myserialports[-1], description='Scale:', disabled=False)
w_pzaxis=widgets.Dropdown(options=myserialports,\
                         value=myserialports[-2], description='Zaxis:', disabled=False)
w_connect.observe(my_connect, names='value')

w_pmove.append(widgets.Button(description='Up 5 mm',button_style='warning'))
w_pmove.append(widgets.Button(description='Up 0.5 mm',button_style='warning'))
w_pmove.append(widgets.Button(description='Down 0.5 mm',button_style='warning'))
w_pmove.append(widgets.Button(description='Down 5 mm',button_style='warning'))
for b in w_pmove:
    b.on_click(zaxis_ctrl)
    

style = {'description_width': 'initial'}
w_nn=widgets.IntSlider(value=nn,min=5,max=50,description='Number of Moves:',style=style)
w_rate=widgets.FloatSlider(value=rate,min=0.1,max=2,step=0.1,description='Frequency (Hz):',style=style)    
w_deltaz=widgets.FloatSlider(value=deltaz,min=0.1,max=2,step=0.1,description='Delta Z (mm):',style=style)
w_bspeed=widgets.IntSlider(value=bspeed,min=50,max=300,description='BSpeed (keep!):',style=style)
w_triggerw=widgets.IntSlider(value=10,min=2,max=100,description='Trigger (g):',style=style)
w_sel_peak=widgets.IntSlider(value=bspeed,min=1,max=50,description='Select Peak:',style=style)
w_start=widgets.Button(description='Start Measurment',button_style='danger')
w_start.on_click(start_measurement)
w_analyse=widgets.Button(description='Analyse Measurment',button_style='info')
w_analyse.on_click(analyse)
w_filename=widgets.Text(description="Filename:",placeholder='Enter your Filename');
w_save=widgets.Button(description='Save Measurement',button_style='info')
w_save.on_click(save_measurement)

w_approach=widgets.Button(description='Do Approach',button_style='primary')
w_approach.on_click(approach)
w_a_label=widgets.Label('******************************')    

out1 = widgets.Output()
with out1:
    plt.plot([0,10],[0,0])
    plt.show()
out2 = widgets.Output()
with out1:
    plt.plot([0,10],[0,0])
    plt.show()

    
display.display(widgets.HBox([widgets.VBox([w_pscale,w_pzaxis]),w_connect]))
display.display(widgets.HBox([widgets.VBox(w_pmove),\
                              widgets.VBox([w_nn,w_rate,w_deltaz,w_bspeed]),\
                              widgets.VBox([w_approach,w_triggerw,w_a_label])]))
display.display(widgets.HBox([widgets.VBox([w_start,w_sel_peak,w_analyse,w_filename,w_save]),out1] ))
display.display(out2)
    


Output()

In [ ]:


nn=10
rate=.5 #frequency of up/down
if ramps:
    speed=140.*rate
else:
    speed=87.*rate
deltaz=1.5

if ~ramps:
    send_printer_marlin('G1 F{:d}\r\n'.format(int(speed*deltaz)).encode())
    
#generate G-code to send to printer
cmd=''
for i in range(nn):
    if ramps:
        cmd +='$J=G91 Z{:4.1f}F{:4.0f};'.format(-deltaz, speed)
        cmd +='$J=G91 Z{:4.1f}F{:4.0f};'.format(deltaz, speed)
    else:
        cmd +='G0 Z{:4.1f};'.format(-deltaz)
        cmd +='G0 Z{:4.1f};'.format(deltaz)

ser_s.write(b'*')  #Tare the scale
ser_s.flushOutput()

time.sleep(1) #stabilize the scale

pt = threading.Thread(target=print_parser, args=(cmd,))
pt.start()
    
%time y,x=read_scale_times(nn*deltaz/rate)
#%time y,x=read_scale_times(nn/rate)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
xnp=np.array(xdata,dtype=np.float32)
ynp=np.array(ydata,dtype=np.float32)
peaks, _ = find_peaks(ydata, height=ynp.max()/2., distance=len(ydata)/2./nn)
plt.plot(xnp, ynp)
for i in range(len(peaks)):
    plt.text(xnp[peaks[i]], ynp[peaks[i]],str(i))

In [ ]:
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import numpy as np

peak_i=-17#peak position to analyse
g=9.81    #gravitational acceleration
obj_h=23. #height in mm
obj_A=np.pi*(13e-3)**2 #Area in m2
delta0=2.5 #initial displacement in mm


down_start_index=int(peaks[peak_i-1]+(peaks[peak_i]-peaks[peak_i-1])/2.)#find down start index
down_end_index=peaks[peak_i]
up_start_index=peaks[peak_i-1]
down_stroke=ynp[down_start_index:down_end_index]
up_stroke=ynp[up_start_index:down_start_index-1]
down_pos=np.linspace(0,deltaz,len(down_stroke))
up_pos=deltaz-np.linspace(0,deltaz,len(up_stroke))
### Calculate strain 
max_strain,_ = integrate.quad(lambda xx: -1./xx, obj_h-delta0, obj_h-delta0-down_pos[-1])
down_strain = np.linspace(0.,max_strain,len(down_stroke))
up_strain = np.linspace(max_strain, 0.,len(up_stroke))

up_stress=up_stroke*1e-3*g/obj_A
down_stress=down_stroke*1e-3*g/obj_A

#Simple Expression for Elastic Modulus, better by fitting a line to the center and take the slope 
EM0=(down_stress[-1]-down_stress[0])/max_strain
plt.text(0,(down_stress[-1]-down_stress[0])/2.+down_stress[0],"E={:5.1f}kPa".format(EM0/1e3))
plt.plot(down_strain,down_stress,label='down');
plt.plot(up_strain,up_stress,label='up')
plt.legend();



In [ ]:
import pickle

with open('Gelatin15DecTue_4b%.pkl', 'wb') as f: 
    pickle.dump([peak_i, g, obj_h, obj_A, delta0, peaks, x, y, xnp, ynp,\
                 deltaz, nn, rate, speed ], f)


In [ ]:
import pickle

with open('PDMS20.pkl', 'rb') as f: 
    peak_i, g, obj_h, obj_A, delta0, peaks, x, y, xnp, ynp,\
                 deltaz, nn, rate, speed = pickle.load(f)

In [ ]:
ramps=False
if ~ramps:
    print('False')

In [ ]:
k=np.array(read_scale(10)).mean()

In [ ]:
k=np.array(read_scale(10)).mean()
print(k>10.)

In [ ]:
trigger


In [ ]:
w_deltaz.value

In [ ]:
display.clear_output()

In [ ]:
out.clear_output()

In [ ]:
out.show()

In [ ]:
peaks

In [ ]:
peaks[-9]

In [11]:
nn=1
nn/2.

0.5